In [ ]:
import os, json, stat
os.makedirs("/root/.kaggle", exist_ok=True)

KAGGLE_USERNAME = "<username>"
KAGGLE_KEY = "<kaggle api key>"

with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump({"username": KAGGLE_USERNAME, "key": KAGGLE_KEY}, f)

os.chmod("/root/.kaggle/kaggle.json", stat.S_IRUSR | stat.S_IWUSR)
print("✅ kaggle.json written with 600 perms")


✅ kaggle.json written with 600 perms


In [ ]:
!pip -q install kaggle

In [ ]:
!kaggle --version
!kaggle datasets list | head

Kaggle API 1.7.4.5
ref                                                                 title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
saidaminsaidaxmadov/chocolate-sales                                 Chocolate Sales                                         468320  2026-01-04 14:23:35.490000          10551        172  1.0              
jayjoshi37/customer-subscription-churn-and-usage-patterns           Customer Subscription Churn and Usage Patterns           34246  2026-01-27 13:53:52.857000            655         23  1.0              
vishardmehta/indian-engineering-college-placement-dataset           Indian Engineering College Placement Dataset            137603  2026-01-24 15:23:40.150000       

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
drive_root = "/content/drive/MyDrive"
data_root = os.path.join(drive_root, "datasets", "ct_to_mri_cgan")
os.makedirs(data_root, exist_ok=True)

!kaggle datasets download darren2020/ct-to-mri-cgan -p "{data_root}" --force
!unzip -o "{data_root}/ct-to-mri-cgan.zip" -d "{data_root}"

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/datasets/ct_to_mri_cgan/Dataset/images/testA/ct1018.png  
  inflating: /content/drive/MyDrive/datasets/ct_to_mri_cgan/Dataset/images/testA/ct1019.png  
  inflating: /content/drive/MyDrive/datasets/ct_to_mri_cgan/Dataset/images/testA/ct1020.png  
  inflating: /content/drive/MyDrive/datasets/ct_to_mri_cgan/Dataset/images/testA/ct1028.png  
  inflating: /content/drive/MyDrive/datasets/ct_to_mri_cgan/Dataset/images/testA/ct1029.png  
  inflating: /content/drive/MyDrive/datasets/ct_to_mri_cgan/Dataset/images/testA/ct1030.png  
  inflating: /content/drive/MyDrive/datasets/ct_to_mri_cgan/Dataset/images/testA/ct1038.png  
  inflating: /content/drive/MyDrive/datasets/ct_to_mri_cgan/Dataset/images/testA/ct1039.png  
  inflating: /content/drive/MyDrive/datasets/ct_to_mri_cgan/Dataset/images/testA/ct1040.png  
  inflating: /content/drive/MyDrive/datasets/ct_to_mri_cgan/Dataset/images/testA/ct1048.png  
  inflati

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
drive_root = "/content/drive/MyDrive"
DATASET_ROOT = os.path.join(drive_root, "datasets", "ct_to_mri_cgan", "Dataset")

print("Dataset root:", DATASET_ROOT)
!ls -lah "{DATASET_ROOT}"
!ls -lah "{DATASET_ROOT}/images" || true
!ls -lah "{DATASET_ROOT}/unseen_demo_images" || true

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset root: /content/drive/MyDrive/datasets/ct_to_mri_cgan/Dataset
total 11K
drwx------ 6 root root 4.0K Feb  1 11:28 images
-rw------- 1 root root 2.3K May 11  2021 sources.txt
drwx------ 4 root root 4.0K Feb  1 11:28 unseen_demo_images
total 16K
drwx------ 2 root root 4.0K Feb  1 11:27 testA
drwx------ 2 root root 4.0K Feb  1 11:27 testB
drwx------ 2 root root 4.0K Feb  1 11:28 trainA
drwx------ 2 root root 4.0K Feb  1 11:28 trainB
total 8.0K
drwx------ 2 root root 4.0K Feb  1 11:28 ct
drwx------ 2 root root 4.0K Feb  1 11:28 mri


In [ ]:
# === Clean, known-good dataset for your folder structure ===
import os, re
from glob import glob
from typing import List, Tuple, Dict
import cv2
import numpy as np
import albumentations as A
import torch
from torch.utils.data import Dataset, DataLoader

# ---- Utilities ----
def _natural_key(s: str):
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r'(\d+)', os.path.basename(s))]

def _pair_by_name_or_order(dir_a: str, dir_b: str) -> List[Tuple[str, str]]:
    exts = (".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff")
    a_paths = sorted([p for ext in exts for p in glob(os.path.join(dir_a, f"*{ext}"))], key=_natural_key)
    b_paths = sorted([p for ext in exts for p in glob(os.path.join(dir_b, f"*{ext}"))], key=_natural_key)
    if not a_paths or not b_paths:
        raise FileNotFoundError(f"No images found in either:\n  {dir_a}\n  {dir_b}")
    def stem(p): return os.path.splitext(os.path.basename(p))[0]
    a_map = {stem(p): p for p in a_paths}
    b_map = {stem(p): p for p in b_paths}
    common = sorted(set(a_map.keys()) & set(b_map.keys()), key=_natural_key)
    if common:
        return [(a_map[k], b_map[k]) for k in common]
    n = min(len(a_paths), len(b_paths))
    if len(a_paths) != len(b_paths):
        print(f"[WARN] Unequal counts ({len(a_paths)} vs {len(b_paths)}). Pairing by order for first {n}.")
    return list(zip(a_paths[:n], b_paths[:n]))

# ---- Normalization ----
def ct_window_normalize(img: np.ndarray, wl: float = 40.0, ww: float = 80.0) -> np.ndarray:
    img = img.astype(np.float32)
    vmin, vmax = wl - ww / 2.0, wl + ww / 2.0
    img = np.clip(img, vmin, vmax)
    return (img - vmin) / max(vmax - vmin, 1e-8)

def mri_robust_normalize(img: np.ndarray, low_q=2.0, high_q=98.0, zscore=False) -> np.ndarray:
    img = img.astype(np.float32)
    if img.max() > 1.0: img = img / 255.0
    lo, hi = np.percentile(img, low_q), np.percentile(img, high_q)
    img = np.clip(img, lo, hi)
    img = (img - lo) / max(hi - lo, 1e-8)
    if zscore:
        img = (img - img.mean()) / (img.std() + 1e-8)
    return img

# ---- MRI bias field aug ----
class RandomBiasField(A.ImageOnlyTransform):
    def __init__(self, bias_range: float = 0.15, kernel_size: int = 81, always_apply=False, p=0.4):
        super().__init__(always_apply, p)
        self.bias_range = bias_range
        self.kernel = kernel_size if kernel_size % 2 == 1 else kernel_size + 1
    def apply(self, img, **params):
        h, w = img.shape[:2]
        low_res = 32
        field = np.random.randn(low_res, low_res).astype(np.float32)
        field = cv2.resize(field, (w, h), interpolation=cv2.INTER_CUBIC)
        field = cv2.GaussianBlur(field, (self.kernel, self.kernel), 0)
        field = (field - field.mean()) / (field.std() + 1e-6)
        field = (field - field.min()) / (field.max() - field.min() + 1e-6)
        field = (1.0 - self.bias_range) + field * (2.0 * self.bias_range)
        out = img * field
        return np.clip(out, 0.0, 1.0)

# ---- Dataset ----
class CTMRIPairedDataset(Dataset):
    """
    split: 'train'  -> Dataset/images/trainA vs trainB
           'test'   -> Dataset/images/testA  vs testB
           'unseen' -> Dataset/unseen_demo_images/ct vs mri
    ct_is_A: True means A=CT and B=MRI. Flip if needed.
    augment: True enables noise/intensity + mild paired spatial transforms.
    """
    def __init__(
        self,
        dataset_root: str,
        split: str = "train",
        size=(256, 256),
        ct_is_A: bool = True,
        ct_window=(40, 80),
        mri_zscore: bool = False,
        augment: bool = True
    ):
        self.size = tuple(size)  # ensure tuple
        self.split = split.lower()
        self.ct_wl, self.ct_ww = ct_window
        self.mri_zscore = mri_zscore
        self.augment = augment

        if self.split in ("train", "test"):
            base = os.path.join(dataset_root, "images")
            a_dir = os.path.join(base, f"{self.split}A")
            b_dir = os.path.join(base, f"{self.split}B")
        elif self.split == "unseen":
            base = os.path.join(dataset_root, "unseen_demo_images")
            a_dir = os.path.join(base, "ct")
            b_dir = os.path.join(base, "mri")
        else:
            raise ValueError("split must be one of: 'train', 'test', 'unseen'")

        if not (os.path.isdir(a_dir) and os.path.isdir(b_dir)):
            raise FileNotFoundError(f"Expected paired folders not found:\n  {a_dir}\n  {b_dir}")

        self.ct_dir, self.mri_dir = (a_dir, b_dir) if ct_is_A else (b_dir, a_dir)
        self.pairs = _pair_by_name_or_order(self.ct_dir, self.mri_dir)

        # Paired spatial transforms
        joint_ops = [A.Resize(height=self.size[0], width=self.size[1], interpolation=cv2.INTER_LINEAR)]
        if self.augment:
            joint_ops += [
                A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=7,
                                   interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_REFLECT_101, p=0.7),
                A.GridDistortion(num_steps=5, distort_limit=0.03, border_mode=cv2.BORDER_REFLECT_101, p=0.3),
            ]
        self.joint_transform = A.Compose(joint_ops, additional_targets={'image2': 'image'}, is_check_shapes=False)

        # Intensity-only augs (safe for correspondence)
        if self.augment:
            self.ct_intensity = A.Compose([
                A.GaussNoise(var_limit=(1e-5, 1e-3), mean=0.0, p=0.35),
                A.RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05, p=0.3),
                A.RandomGamma(gamma_limit=(90, 110), p=0.3),
            ])
            self.mri_intensity = A.Compose([
                A.GaussNoise(var_limit=(1e-5, 1e-3), mean=0.0, p=0.4),
                A.RandomGamma(gamma_limit=(80, 120), p=0.4),
                RandomBiasField(bias_range=0.15, kernel_size=81, p=0.4),
            ])
        else:
            self.ct_intensity = A.Compose([])
            self.mri_intensity = A.Compose([])

    def __len__(self): return len(self.pairs)

    def _read_gray(self, path: str) -> np.ndarray:
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        if img is None:
            raise FileNotFoundError(f"Cannot read image: {path}")
        if img.ndim == 3: img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return img.astype(np.float32)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        ct_path, mri_path = self.pairs[idx]
        # A) load
        ct_img  = self._read_gray(ct_path)
        mri_img = self._read_gray(mri_path)
        # B) paired spatial ops (resize + mild jitter)
        t = self.joint_transform(image=ct_img, image2=mri_img)
        ct_img, mri_img = t['image'], t['image2']
        # C) independent normalization
        ct_base  = ct_window_normalize(ct_img, wl=self.ct_wl, ww=self.ct_ww)
        mri_base = mri_robust_normalize(mri_img, low_q=2, high_q=98, zscore=False)
        # D) intensity variations (no geometry)
        ct_aug  = np.clip(self.ct_intensity(image=ct_base)['image'], 0.0, 1.0)
        mri_aug = np.clip(self.mri_intensity(image=mri_base)['image'], 0.0, 1.0)
        if self.mri_zscore:
            mri_aug = (mri_aug - mri_aug.mean()) / (mri_aug.std() + 1e-8)
        # E) tensors (C,H,W)
        return {
            "ct": torch.from_numpy(ct_aug).unsqueeze(0).float(),
            "mri": torch.from_numpy(mri_aug).unsqueeze(0).float(),
            "ct_path": ct_path, "mri_path": mri_path
        }

# ---- Loader ----
def make_loader(
    dataset_root: str,
    split: str = "train",
    size=(256, 256),
    batch_size=8,
    num_workers=2,
    shuffle=True,
    **kwargs
):
    ds = CTMRIPairedDataset(dataset_root=dataset_root, split=split, size=size, **kwargs)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True)
    return ds, dl


In [ ]:
import os
drive_root = "/content/drive/MyDrive"
DATASET_ROOT = os.path.join(drive_root, "datasets", "ct_to_mri_cgan", "Dataset")

# TRAIN loader with augmentations ON
train_ds, train_dl = make_loader(
    dataset_root=DATASET_ROOT,
    split="train",           # 'train' -> images/trainA vs trainB
    size=(256, 256),
    batch_size=8,
    ct_is_A=True,            # set False if you find A is MRI, B is CT
    ct_window=(40, 80),
    mri_zscore=False,
    shuffle=True,
    augment=True             # ✅ augmentations enabled during training
)

# TEST/VAL loader with augmentations OFF (deterministic)
val_ds, val_dl = make_loader(
    dataset_root=DATASET_ROOT,
    split="test",            # 'test' -> images/testA vs testB
    size=(256, 256),
    batch_size=8,
    ct_is_A=True,
    augment=False            # ❌ no aug for evaluation
)

# UNSEEN (small demo set)
unseen_ds, unseen_dl = make_loader(
    dataset_root=DATASET_ROOT,
    split="unseen",          # unseen_demo_images/ct vs mri
    size=(256, 256),
    batch_size=4,
    ct_is_A=True,
    augment=False
)

# Peek a batch
batch = next(iter(train_dl))
print("CT:", batch['ct'].shape, batch['ct'].min().item(), batch['ct'].max().item())
print("MRI:", batch['mri'].shape, batch['mri'].min().item(), batch['mri'].max().item())

[WARN] Unequal counts (1742 vs 1744). Pairing by order for first 1742.


/tmp/ipython-input-3389951250.py:122: UserWarning: Argument(s) 'var_limit, mean' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(1e-5, 1e-3), mean=0.0, p=0.35),
/tmp/ipython-input-3389951250.py:127: UserWarning: Argument(s) 'var_limit, mean' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(1e-5, 1e-3), mean=0.0, p=0.4),


CT: torch.Size([8, 1, 256, 256]) 0.0 1.0
MRI: torch.Size([8, 1, 256, 256]) 0.0 1.0


# Feature Extraction

In [ ]:
import math
from dataclasses import dataclass
from typing import Tuple, Dict, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler

In [ ]:
# Convolutional encoder/decoder
class ConvEncoder(nn.Module):
    """
    Simple 2D conv encoder → latent vector.
    in_ch=1 for grayscale CT/MRI. Latent is a vector (B, d_latent).
    """
    def __init__(self, in_ch: int = 1, d_latent: int = 256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, 32, 3, 2, 1), nn.BatchNorm2d(32), nn.ReLU(inplace=True),   # 1/2
            nn.Conv2d(32, 64, 3, 2, 1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),      # 1/4
            nn.Conv2d(64, 128, 3, 2, 1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),    # 1/8
            nn.Conv2d(128, 256, 3, 2, 1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),   # 1/16
        )
        self.proj = nn.Linear(256, d_latent)

    def forward(self, x):
        f = self.net(x)                    # (B, 256, H/16, W/16)
        f = f.mean(dim=(2,3))              # global avg pool → (B, 256)
        z = self.proj(f)                   # (B, d_latent)
        return z


class ConvDecoder(nn.Module):
    """
    Latent vector → 2D reconstruction (1×H×W).
    Uses a small MLP to seed a 1/16 feature map then up-convs to full size.
    """
    def __init__(self, d_latent: int = 256, out_ch: int = 1, base_hw: Tuple[int, int] = (16, 16)):
        super().__init__()
        self.base_hw = base_hw
        self.fc = nn.Linear(d_latent, 256 * base_hw[0] * base_hw[1])
        self.up = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),  # x2
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),    # x4
            nn.ConvTranspose2d(64, 32, 4, 2, 1), nn.BatchNorm2d(32), nn.ReLU(inplace=True),     # x8
            nn.ConvTranspose2d(32, out_ch, 4, 2, 1), nn.Sigmoid()                                # x16 → [0,1]
        )

    def forward(self, z):
        B = z.size(0)
        x = self.fc(z).view(B, 256, self.base_hw[0], self.base_hw[1])
        x = self.up(x)  # (B, 1, H, W)
        return x

In [ ]:
#VAE Head
class VAEEncoderHead(nn.Module):
    """Projects encoder features into μ and logσ² for a Gaussian posterior."""
    def __init__(self, in_dim: int, d_latent: int):
        super().__init__()
        self.mu = nn.Linear(in_dim, d_latent)
        self.logvar = nn.Linear(in_dim, d_latent)

    def forward(self, h):
        mu = self.mu(h)
        logvar = self.logvar(h)
        return mu, logvar


def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return mu + eps * std

In [ ]:
#ViT-style Encoder
class PatchEmbed(nn.Module):
    """Split image into non-overlapping patches and project to embedding dim."""
    def __init__(self, in_ch=1, embed_dim=256, patch=16):
        super().__init__()
        self.proj = nn.Conv2d(in_ch, embed_dim, kernel_size=patch, stride=patch)

    def forward(self, x):
        x = self.proj(x)                # (B, E, H/P, W/P)
        B, E, Hp, Wp = x.shape
        x = x.flatten(2).transpose(1, 2)  # (B, N, E), where N=Hp*Wp
        return x, (Hp, Wp)


class ViTEncoder(nn.Module):
    """
    Minimal ViT encoder: patch embedding → TransformerEncoder → CLS pooling.
    """
    def __init__(self, in_ch=1, embed_dim=256, depth=4, heads=8, mlp_ratio=4.0, patch=16, d_latent=256, dropout=0.0):
        super().__init__()
        self.patch = patch
        self.patch_embed = PatchEmbed(in_ch, embed_dim, patch=patch)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=heads, dim_feedforward=int(embed_dim * mlp_ratio),
            dropout=dropout, batch_first=True, activation='gelu'
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=depth)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = None  # simple; we’ll rely on content (ok for small depth)
        self.to_latent = nn.Linear(embed_dim, d_latent)

        nn.init.trunc_normal_(self.cls_token, std=0.02)

    def forward(self, x):
        B = x.size(0)
        x, (Hp, Wp) = self.patch_embed(x)        # (B, N, E)
        cls = self.cls_token.expand(B, -1, -1)   # (B, 1, E)
        x = torch.cat([cls, x], dim=1)           # prepend CLS
        x = self.encoder(x)                      # (B, 1+N, E)
        cls_out = x[:, 0]                        # (B, E)
        z = self.to_latent(cls_out)              # (B, d_latent)
        return z

In [ ]:
#Helpers
class MLP(nn.Module):
    def __init__(self, d_in, d_out, hidden=0):
        super().__init__()
        if hidden > 0:
            self.net = nn.Sequential(
                nn.Linear(d_in, hidden), nn.ReLU(inplace=True),
                nn.Linear(hidden, d_out)
            )
        else:
            self.net = nn.Linear(d_in, d_out)
    def forward(self, x):
        return self.net(x)

In [ ]:
#CAE based multimodal autoencoder
class MMCAE(nn.Module):
    """
    Multi-Modal Convolutional Autoencoder:
    - Modality-specific encoders (ConvEncoder)
    - Projection heads to shared latent (z_ct, z_mri)
    - Modality-specific decoders (ConvDecoder) that decode from shared z
    """
    def __init__(self, d_latent=256, H=256, W=256):
        super().__init__()
        self.enc_ct  = ConvEncoder(in_ch=1, d_latent=d_latent)
        self.enc_mri = ConvEncoder(in_ch=1, d_latent=d_latent)

        # small projectors (can be identity)
        self.proj_ct  = MLP(d_latent, d_latent, hidden=0)
        self.proj_mri = MLP(d_latent, d_latent, hidden=0)

        base_hw = (H // 16, W // 16)
        self.dec_ct  = ConvDecoder(d_latent, out_ch=1, base_hw=base_hw)
        self.dec_mri = ConvDecoder(d_latent, out_ch=1, base_hw=base_hw)

    def forward(self, ct, mri):
        # Encode
        h_ct  = self.enc_ct(ct)
        h_mri = self.enc_mri(mri)

        # Shared latent
        z_ct  = self.proj_ct(h_ct)
        z_mri = self.proj_mri(h_mri)

        # Recon (self)
        x_ct_rec  = self.dec_ct(z_ct)
        x_mri_rec = self.dec_mri(z_mri)

        # Cross-recon
        x_ct_from_mri  = self.dec_ct(z_mri)
        x_mri_from_ct  = self.dec_mri(z_ct)

        return {
            "z_ct": z_ct, "z_mri": z_mri,
            "x_ct_rec": x_ct_rec, "x_mri_rec": x_mri_rec,
            "x_ct_from_mri": x_ct_from_mri, "x_mri_from_ct": x_mri_from_ct
        }

In [ ]:
#VAE-based multimodal autoencoder
class MMVAE(nn.Module):
    """
    Multi-Modal VAE:
    - Modality-specific encoders produce (μ, logσ²)
    - Sample z_ct, z_mri
    - Modality-specific decoders decode from shared z
    - Supports self-reconstruction + cross-reconstruction + KL
    """
    def __init__(self, d_latent=256, H=256, W=256):
        super().__init__()
        # We'll share the ConvEncoder body but tap pooled features
        self.back_ct  = nn.Sequential(
            nn.Conv2d(1,32,3,2,1), nn.ReLU(),
            nn.Conv2d(32,64,3,2,1), nn.ReLU(),
            nn.Conv2d(64,128,3,2,1), nn.ReLU(),
            nn.Conv2d(128,256,3,2,1), nn.ReLU()
        )
        self.back_mri = nn.Sequential(
            nn.Conv2d(1,32,3,2,1), nn.ReLU(),
            nn.Conv2d(32,64,3,2,1), nn.ReLU(),
            nn.Conv2d(64,128,3,2,1), nn.ReLU(),
            nn.Conv2d(128,256,3,2,1), nn.ReLU()
        )
        self.head_ct  = VAEEncoderHead(256, d_latent)
        self.head_mri = VAEEncoderHead(256, d_latent)

        base_hw = (H // 16, W // 16)
        self.dec_ct  = ConvDecoder(d_latent, out_ch=1, base_hw=base_hw)
        self.dec_mri = ConvDecoder(d_latent, out_ch=1, base_hw=base_hw)

    def encode(self, x, backbone, head):
        h = backbone(x).mean(dim=(2,3))  # GAP → (B, 256)
        mu, logvar = head(h)
        z = reparameterize(mu, logvar)
        return z, mu, logvar

    def forward(self, ct, mri):
        z_ct, mu_ct, lv_ct   = self.encode(ct,  self.back_ct,  self.head_ct)
        z_mri, mu_mri, lv_mri= self.encode(mri, self.back_mri, self.head_mri)

        x_ct_rec  = self.dec_ct(z_ct)
        x_mri_rec = self.dec_mri(z_mri)
        x_ct_from_mri = self.dec_ct(z_mri)
        x_mri_from_ct = self.dec_mri(z_ct)

        return {
            "z_ct": z_ct, "z_mri": z_mri,
            "mu_ct": mu_ct, "logvar_ct": lv_ct,
            "mu_mri": mu_mri, "logvar_mri": lv_mri,
            "x_ct_rec": x_ct_rec, "x_mri_rec": x_mri_rec,
            "x_ct_from_mri": x_ct_from_mri, "x_mri_from_ct": x_mri_from_ct
        }

In [ ]:
#ViT-encode + conv decoder
class MMViT(nn.Module):
    """
    Multi-Modal with Transformer encoders:
    - Modality-specific ViT encoders → shared latent
    - Modality-specific Conv decoders from shared z
    """
    def __init__(self, d_latent=256, H=256, W=256, patch=16):
        super().__init__()
        self.enc_ct  = ViTEncoder(in_ch=1, embed_dim=256, depth=4, heads=8, patch=patch, d_latent=d_latent)
        self.enc_mri = ViTEncoder(in_ch=1, embed_dim=256, depth=4, heads=8, patch=patch, d_latent=d_latent)

        base_hw = (H // 16, W // 16)
        self.dec_ct  = ConvDecoder(d_latent, out_ch=1, base_hw=base_hw)
        self.dec_mri = ConvDecoder(d_latent, out_ch=1, base_hw=base_hw)

    def forward(self, ct, mri):
        z_ct  = self.enc_ct(ct)
        z_mri = self.enc_mri(mri)

        x_ct_rec  = self.dec_ct(z_ct)
        x_mri_rec = self.dec_mri(z_mri)
        x_ct_from_mri = self.dec_ct(z_mri)
        x_mri_from_ct = self.dec_mri(z_ct)

        return {
            "z_ct": z_ct, "z_mri": z_mri,
            "x_ct_rec": x_ct_rec, "x_mri_rec": x_mri_rec,
            "x_ct_from_mri": x_ct_from_mri, "x_mri_from_ct": x_mri_from_ct
        }

In [ ]:
# Losses: self-recon, cross-recon, latent alignment (+KL for VAE)
@dataclass
class LossWeights:
    recon: float = 1.0          # self-reconstruction L1
    cross_recon: float = 0.5    # cross reconstruction L1
    align: float = 0.2          # latent alignment (MSE)
    kl: float = 0.05            # only used for VAE models
    info_nce: float = 0.0       # optional contrastive (set >0 to enable)


def l1_recon(pred, target):
    return F.l1_loss(pred, target)


def latent_align_mse(z_ct, z_mri):
    return F.mse_loss(z_ct, z_mri)


def kld_loss(mu, logvar):
    # KL divergence to N(0, I):  -0.5 * sum(1 + logσ² - μ² - σ²)
    return -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())


def info_nce(z_ct, z_mri, temperature=0.1):
    """
    Simple symmetric InfoNCE using cosine similarities.
    Assumes z_ct and z_mri are (B, D) and pairs i match across modalities.
    """
    z1 = F.normalize(z_ct, dim=-1)
    z2 = F.normalize(z_mri, dim=-1)
    logits = z1 @ z2.t() / temperature               # (B, B)
    labels = torch.arange(z1.size(0), device=z1.device)
    loss = F.cross_entropy(logits, labels) + F.cross_entropy(logits.t(), labels)
    return loss * 0.5


In [ ]:
#Training Loop
def build_model(backbone: str = "cae", H: int = 256, W: int = 256, d_latent: int = 256):
    backbone = backbone.lower()
    if backbone == "cae":
        return MMCAE(d_latent=d_latent, H=H, W=W)
    elif backbone == "vae":
        return MMVAE(d_latent=d_latent, H=H, W=W)
    elif backbone in ("vit", "transformer"):
        return MMViT(d_latent=d_latent, H=H, W=W, patch=16)
    else:
        raise ValueError("backbone must be one of: cae, vae, vit")


def train_epoch(model, loader, optimizer, device, weights: LossWeights, is_vae: bool):
    model.train()
    scaler = GradScaler(enabled=torch.cuda.is_available())
    running = {"loss": 0.0, "recon": 0.0, "cross": 0.0, "align": 0.0, "kl": 0.0, "n": 0}

    for batch in loader:
        ct  = batch["ct"].to(device)    # (B,1,H,W), range [0,1] as per preprocessing
        mri = batch["mri"].to(device)

        optimizer.zero_grad(set_to_none=True)
        with autocast(enabled=torch.cuda.is_available()):
            out = model(ct, mri)
            # self recon
            loss_recon = l1_recon(out["x_ct_rec"], ct) + l1_recon(out["x_mri_rec"], mri)

            # cross recon
            loss_cross = l1_recon(out["x_ct_from_mri"], ct) + l1_recon(out["x_mri_from_ct"], mri)

            # latent align (shared latent consistency)
            loss_align = latent_align_mse(out["z_ct"], out["z_mri"])

            # KL (VAE only)
            loss_kl = torch.tensor(0.0, device=device)
            if is_vae:
                loss_kl = kld_loss(out["mu_ct"], out["logvar_ct"]) + kld_loss(out["mu_mri"], out["logvar_mri"])

            # optional InfoNCE
            loss_nce = torch.tensor(0.0, device=device)
            if weights.info_nce > 0.0:
                loss_nce = info_nce(out["z_ct"], out["z_mri"])

            loss = (weights.recon * loss_recon +
                    weights.cross_recon * loss_cross +
                    weights.align * loss_align +
                    weights.kl * loss_kl +
                    weights.info_nce * loss_nce)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        bs = ct.size(0)
        running["loss"]  += loss.item() * bs
        running["recon"] += loss_recon.item() * bs
        running["cross"] += loss_cross.item() * bs
        running["align"] += loss_align.item() * bs
        running["kl"]    += (loss_kl.item() if torch.is_tensor(loss_kl) else 0.0) * bs
        running["n"]     += bs

    for k in ["loss", "recon", "cross", "align", "kl"]:
        running[k] /= max(running["n"], 1)
    return running


@torch.no_grad()
def eval_epoch(model, loader, device, is_vae: bool):
    model.eval()
    running = {"loss": 0.0, "recon": 0.0, "cross": 0.0, "align": 0.0, "kl": 0.0, "n": 0}
    for batch in loader:
        ct  = batch["ct"].to(device)
        mri = batch["mri"].to(device)
        out = model(ct, mri)
        loss_recon = l1_recon(out["x_ct_rec"], ct) + l1_recon(out["x_mri_rec"], mri)
        loss_cross = l1_recon(out["x_ct_from_mri"], ct) + l1_recon(out["x_mri_from_ct"], mri)
        loss_align = latent_align_mse(out["z_ct"], out["z_mri"])
        loss_kl = torch.tensor(0.0, device=device)
        if is_vae:
            loss_kl = kld_loss(out["mu_ct"], out["logvar_ct"]) + kld_loss(out["mu_mri"], out["logvar_mri"])
        loss = loss_recon + 0.5*loss_cross + 0.2*loss_align + 0.05*loss_kl

        bs = ct.size(0)
        running["loss"]  += loss.item() * bs
        running["recon"] += loss_recon.item() * bs
        running["cross"] += loss_cross.item() * bs
        running["align"] += loss_align.item() * bs
        running["kl"]    += (loss_kl.item() if torch.is_tensor(loss_kl) else 0.0) * bs
        running["n"]     += bs

    for k in ["loss", "recon", "cross", "align", "kl"]:
        running[k] /= max(running["n"], 1)
    return running

In [ ]:
#Train model
# Choose backbone: "cae", "vae", or "vit"
backbone = "cae"        # or "vae", "vit"
H, W = 256, 256
d_latent = 256

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_model(backbone, H=H, W=W, d_latent=d_latent).to(device)

is_vae = (backbone == "vae")
weights = LossWeights(recon=1.0, cross_recon=0.5, align=0.2, kl=0.05 if is_vae else 0.0, info_nce=0.0)

optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, betas=(0.5, 0.999), weight_decay=0.0)

EPOCHS = 20
for epoch in range(1, EPOCHS+1):
    tr = train_epoch(model, train_dl, optimizer, device, weights, is_vae)
    va = eval_epoch(model,  val_dl,   device, is_vae)
    print(f"[{epoch:03d}] "
          f"train loss={tr['loss']:.4f} (recon {tr['recon']:.4f}, cross {tr['cross']:.4f}, align {tr['align']:.4f}, kl {tr['kl']:.4f}) | "
          f"val loss={va['loss']:.4f}")

# Save weights
torch.save(model.state_dict(), f"mm_{backbone}_d{d_latent}_hw{H}x{W}.pt")

/tmp/ipython-input-2995713958.py:16: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=torch.cuda.is_available())
/tmp/ipython-input-2995713958.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=torch.cuda.is_available()):


[001] train loss=0.6036 (recon 0.3897, cross 0.4246, align 0.0082, kl 0.0000) | val loss=0.4204


KeyboardInterrupt: 

# Unsupervised Anomaly Detection

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from typing import Dict, Tuple
from dataclasses import dataclass

@torch.no_grad()
def forward_pass_collect(model, loader, device, is_vae=False):
    """
    Runs the model over a loader and collects:
    - z_ct, z_mri (shared latents)
    - reconstructions (self + cross)
    - (VAE only) mu/logvar for each modality
    Also returns per-sample L1 errors for recon and cross-recon.
    """
    model.eval()
    Z_ct, Z_mri = [], []
    MU_ct, LV_ct, MU_mri, LV_mri = [], [], [], []
    re_ct, re_mri, cre_ct, cre_mri = [], [], [], []

    for batch in loader:
        ct  = batch["ct"].to(device)
        mri = batch["mri"].to(device)
        out = model(ct, mri)

        # Latents
        Z_ct.append(out["z_ct"].cpu())
        Z_mri.append(out["z_mri"].cpu())

        # Recon errors (mean per sample)
        re_ct.append(F.l1_loss(out["x_ct_rec"], ct, reduction='none').mean(dim=(1,2,3)).cpu())
        re_mri.append(F.l1_loss(out["x_mri_rec"], mri, reduction='none').mean(dim=(1,2,3)).cpu())
        cre_ct.append(F.l1_loss(out["x_ct_from_mri"], ct, reduction='none').mean(dim=(1,2,3)).cpu())
        cre_mri.append(F.l1_loss(out["x_mri_from_ct"], mri, reduction='none').mean(dim=(1,2,3)).cpu())

        if is_vae:
            MU_ct.append(out["mu_ct"].cpu()); LV_ct.append(out["logvar_ct"].cpu())
            MU_mri.append(out["mu_mri"].cpu()); LV_mri.append(out["logvar_mri"].cpu())

    Z_ct  = torch.cat(Z_ct, dim=0)
    Z_mri = torch.cat(Z_mri, dim=0)
    re_ct  = torch.cat(re_ct, dim=0)
    re_mri = torch.cat(re_mri, dim=0)
    cre_ct = torch.cat(cre_ct, dim=0)
    cre_mri= torch.cat(cre_mri, dim=0)

    pack = {
        "z_ct": Z_ct, "z_mri": Z_mri,
        "re_ct": re_ct, "re_mri": re_mri,
        "cre_ct": cre_ct, "cre_mri": cre_mri
    }
    if is_vae:
        pack.update({
            "mu_ct": torch.cat(MU_ct, dim=0),
            "logvar_ct": torch.cat(LV_ct, dim=0),
            "mu_mri": torch.cat(MU_mri, dim=0),
            "logvar_mri": torch.cat(LV_mri, dim=0),
        })
    return pack

def to_numpy(t: torch.Tensor) -> np.ndarray:
    return t.detach().cpu().numpy()

In [ ]:
@dataclass
class ReconWeights:
    self_recon_w: float = 0.6   # weight for CT/MRI self recon
    cross_recon_w: float = 0.4  # weight for cross-modal recon errors

def make_autoencoder_scores(pack: Dict[str, torch.Tensor], w: ReconWeights):
    """
    Produces a per-sample anomaly score:
    score = w_self * mean(self-recon errors) + w_cross * mean(cross-recon errors)
    All errors are L1 averaged per-sample.
    """
    re_self  = 0.5 * (pack["re_ct"] + pack["re_mri"])         # (N,)
    re_cross = 0.5 * (pack["cre_ct"] + pack["cre_mri"])       # (N,)
    score = w.self_recon_w * re_self + w.cross_recon_w * re_cross
    return score, {"re_self": re_self, "re_cross": re_cross}

# Example usage after training:
# train_pack = forward_pass_collect(model, train_dl, device, is_vae=(backbone=="vae"))
# test_pack  = forward_pass_collect(model,  val_dl, device, is_vae=(backbone=="vae"))
# scores_train_AE, parts_train = make_autoencoder_scores(train_pack, ReconWeights())
# scores_test_AE,  parts_test  = make_autoencoder_scores(test_pack,  ReconWeights())